# [Module 5.1] getRec() 통한 추천 리스트 얻기

이 노트북은 Module2에서 생성한 솔류션을 바탕으로 아래와 같은 작업을 합니다.
* 캠페인 생성
* 캠페인을 통해 특정 유저에 대한 추천 영화 리스트 얻기


In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 샘플 추천 결과 얻기

캠페인이 활성화되면 추천 결과를 받을 수 있습니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음, ID 대신 추천을 위해 영화 정보를 표시하는 몇 가지 헬퍼 함수를 만듭니다.

In [19]:
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()

items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
items.tail()

#len(unique_items)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,ITEM_ID,TITLE,GENRE
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [20]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())


#### HRNN GetRecommendations 호출

아래 코드 셀을 실행하면 특정 사용자에 대한 추천 사항이 표시되고 추천 영화 목록이 반환됩니다.

In [6]:
# df=pd.read_csv(interaction_filename)

# # Getting a random user:
# user_id, item_id, _,_,_ = df.sample().values[0]
# print("USER: {}".format(user_id))

# get_recommendations_response = personalize_runtime.get_recommendations(
#     campaignArn = hrnn_campaign_arn,
#     userId = str(user_id),
# )
# # Update DF rendering
# pd.set_option('display.max_rows', 30)

# print("Recommendations for user: ", user_id)

# item_list = get_recommendations_response['itemList']
# recommendation_title_list = []
# recommendation_id_list=[]
# for item in item_list:
#     title = get_movie_title(item['itemId'])
#     recommendation_title_list.append(title)
#     recommendation_id_list.append(item['itemId'])
# recommendations_df = pd.DataFrame(recommendation_title_list, columns = ['OriginalRecs'])
# recommendations_df

#### Sims GetRecommendations 호출
아래 코드 셀을 실행하면 특정 아이템과 유사한 추천 영화 목록이 반환됩니다.

In [7]:
# # Getting a random user:
# user_id, item_id, _,_,_ = df.sample().values[0]
# print("ITEM ID: {}".format(item_id))


# get_recommendations_response = personalize_runtime.get_recommendations(
#     campaignArn = sims_campaign_arn,
#     itemId = str(item_id),
# )
# # Update DF rendering
# pd.set_option('display.max_rows', 30)

# print("Recommendations for item_id: ", item_id)

# item_list = get_recommendations_response['itemList']
# recommendation_title_list = []
# recommendation_id_list=[]
# for item in item_list:
#     title = get_movie_title(item['itemId'])
#     recommendation_title_list.append(title)
#     recommendation_id_list.append(item['itemId'])
# recommendations_df = pd.DataFrame(recommendation_title_list, columns = ['OriginalRecs'])
# recommendations_df

## Amazon Personalize Batch Export 작업 


Amazon Personalize Batch기능을 활용하려면 json 파일 형식으로 추천 받고하 자는 사용자 또는 아이템 아이디를 json 형태의 파일로 s3에 저장하여야 합니다. Output의 형식도 json형태로 저장되며 지정한 S3 bucket 경로에 저장 되게 됩니다. 

HRNN 솔루션  Batch Input 예제: 

```JSON,
    {"userId": "4638"},
    {"userId": "663"},
    {"userId": "3384"},
```


Batch Output 예제: 
```JSON,
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```


In [8]:
#Get the user list
df=pd.read_csv(warm_train_interaction_filename)
df_users = df['USER_ID'].unique()
df_users=pd.DataFrame(df_users,columns=['USER_ID'])


In [9]:
#Get the user list
#batch_users = df_users.sample(3).index.tolist()
batch_users=df_users.index.tolist()
data_dir="dataset/"
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir+json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [10]:
# Showcase the input file:
!head -n5 $data_dir$json_input_filename

{"userId": "0"}
{"userId": "1"}
{"userId": "2"}
{"userId": "3"}
{"userId": "4"}


In [11]:
!tail -n5 $data_dir$json_input_filename

{"userId": "6035"}
{"userId": "6036"}
{"userId": "6037"}
{"userId": "6038"}
{"userId": "6039"}


In [12]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket).Object(data_dir+json_input_filename).upload_file(data_dir+json_input_filename)
s3_input_path = "s3://" + bucket + "/" + data_dir+json_input_filename
print(s3_input_path)

s3://sagemaker-ap-northeast-2-057716757052/dataset/json_input.json


In [13]:
# Define the output path
s3_output_path = "s3://" + bucket + "/"+data_dir
print(s3_output_path)

s3://sagemaker-ap-northeast-2-057716757052/dataset/


In [14]:
print(role_arn)

arn:aws:iam::057716757052:role/PersonalizeRoleDemo91891


In [15]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = hrnn_solution_version_arn,
    jobName = "POC-Batch-Inference-Job-HRNN-"+suffix,
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

In [16]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  12:58:25 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [21]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket,data_dir+export_name,data_dir+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open("dataset/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 1024,User: 1029,User: 1064,User: 1080,User: 110,User: 1102,User: 1134,User: 1173,User: 1207,User: 1232,...,User: 5856,User: 5966,User: 5971,User: 676,User: 697,User: 803,User: 806,User: 810,User: 879,User: 905
0,"Purple Rose of Cairo, The (1985)",Weird Science (1985),Rising Sun (1993),Cape Fear (1962),Mission to Mars (2000),Amadeus (1984),Indiana Jones and the Temple of Doom (1984),Die Hard: With a Vengeance (1995),"Exorcist, The (1973)",Trading Places (1983),...,Top Gun (1986),Howards End (1992),Die Hard 2 (1990),"Honey, I Shrunk the Kids (1989)",Dave (1993),Star Wars: Episode I - The Phantom Menace (1999),Deliverance (1972),Groundhog Day (1993),"Sting, The (1973)",Apollo 13 (1995)
1,Sabrina (1954),Back to the Future Part II (1989),Lethal Weapon 4 (1998),"Man Who Knew Too Much, The (1956)",Scream 3 (2000),To Kill a Mockingbird (1962),Crocodile Dundee (1986),Backdraft (1991),"American Werewolf in London, An (1981)",Sixteen Candles (1984),...,"Princess Bride, The (1987)",Toy Story (1995),Get Shorty (1995),"'burbs, The (1989)",To Die For (1995),True Lies (1994),Fitzcarraldo (1982),Toy Story 2 (1999),Annie Hall (1977),Jerry Maguire (1996)
2,Jerry Maguire (1996),Adventures in Babysitting (1987),Far and Away (1992),"Boys from Brazil, The (1978)",Sphere (1998),Pulp Fiction (1994),Con Air (1997),Outbreak (1995),Gremlins (1984),"Return of the Pink Panther, The (1974)",...,Star Trek IV: The Voyage Home (1986),"English Patient, The (1996)",Romancing the Stone (1984),Police Academy (1984),L.A. Story (1991),Crocodile Dundee (1986),Singin' in the Rain (1952),Clerks (1994),Duck Soup (1933),"Fisher King, The (1991)"
3,Prizzi's Honor (1985),Night Shift (1982),G.I. Jane (1997),Lord of the Flies (1963),Reindeer Games (2000),Seven Samurai (The Magnificent Seven) (Shichin...,Top Gun (1986),"Crow, The (1994)",Friday the 13th (1980),Big (1988),...,Romancing the Stone (1984),"Remains of the Day, The (1993)",Superman (1978),Back to the Future Part III (1990),As Good As It Gets (1997),Galaxy Quest (1999),Amadeus (1984),Toy Story (1995),Toy Story 2 (1999),Quiz Show (1994)
4,Four Weddings and a Funeral (1994),Married to the Mob (1988),"Devil's Own, The (1997)",Big (1988),Shaft (2000),"Grapes of Wrath, The (1940)","NeverEnding Story, The (1984)",Broken Arrow (1996),Young Frankenstein (1974),Get Shorty (1995),...,Star Wars: Episode VI - Return of the Jedi (1983),Smoke Signals (1998),Indiana Jones and the Temple of Doom (1984),"Mighty Ducks, The (1992)",Men in Black (1997),Starship Troopers (1997),"Boat, The (Das Boot) (1981)",M*A*S*H (1970),"Shop Around the Corner, The (1940)","River Runs Through It, A (1992)"
5,Witness (1985),Back to School (1986),Backdraft (1991),Foreign Correspondent (1940),What Lies Beneath (2000),"Maltese Falcon, The (1941)","Goonies, The (1985)","World Is Not Enough, The (1999)",Frankenstein (1931),Wayne's World (1992),...,Total Recall (1990),Shall We Dance? (Shall We Dansu?) (1996),Tomorrow Never Dies (1997),Back to School (1986),Four Weddings and a Funeral (1994),Romancing the Stone (1984),Ben-Hur (1959),Annie Hall (1977),Network (1976),Nobody's Fool (1994)
6,"Next Stop, Wonderland (1998)",Arthur (1981),Kiss the Girls (1997),Witness (1985),"Beach, The (2000)",Citizen Kane (1941),Easy Rider (1969),Conspiracy Theory (1997),Fright Night (1985),Austin Powers: International Man of Mystery (1...,...,Men in Black (1997),"Postino, Il (The Postman) (1994)",Top Gun (1986),Erin Brockovich (2000),Smoke Signals (1998),"Mummy, The (1999)",Psycho (1960),This Is Spinal Tap (1984),Groundhog Day (1993),Chasing Amy (1997)
7,Titanic (1997),"Man with Two Brains, The (1983)",Midnight in the Garden of Good and Evil (1997),Chinatown (1974),Gossip (2000),Apocalypse Now (1979),Escape from New York (1981),Natural Born Killers (1994),Dracula (1931),Lethal Weapon (1987),...,Indiana Jones and the Temple of Doom (1984),Big Night (1996),"Spy Who Loved Me, The (1977)",Arachnophobia (1990),"League of Their Own, A (1992)",Con Air (1997),Citizen Kane (1941),M

## 리뷰

캠페인을 생성하고 실제적으로 특정 유저의 추천 영화 목록도 얻었습니다.
이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`4.View_Campaign_And_Interactions.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [ ]:
%store recommendations_df
%store bulk_recommendations_df
%store user_id